In [1]:
import pandas as pd
import csv
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Read in data and add the column names
recipes = pd.read_csv('../data/clean_recipes.csv')
recipes.columns=['recipe_id', 'recipe_tag', 'title', 'rating', 'total_reviews', 'ingredients', 'calories', 'carbs',\
                'protein', 'cholesterol', 'sodium']
recipes.head()

,recipe_id,recipe_tag,title,rating,total_reviews,ingredients,calories,carbs,protein,cholesterol,sodium
0,1,italian,Italian Chicken Cacciatore,5.00,3,"['chicken', 'pepper', 'dry white wine', 's tom...",587,36.5,38.8,117,1707
1,2,italian,Spaghetti Cacio e Pepe,4.58,73,"['pepper', ""Pecorino Romano cheese'"", 'olive o...",806,87.8,31.6,54,633
2,3,italian,Easy Caponata,4.67,3,"['black olives', 'pepper', ' eggplant', 'onion...",289,25.2,4.4,0,424
3,4,italian,Chef John's Lemon Ice,4.64,10,"['water', ""lemon extract'"", 'ly squeezed lemon...",94,24.8,0.1,0,4
4,5,italian,Focaccia di Recco,4.60,17,"['water', 'quarter-sheet baking pan', 'extra-v...",186,24.1,4.7,7,327


In [3]:
# Process the data and get a count of how many times an ingredient occurs in the recipes
recipes['ingredients'] = recipes['ingredients'].apply(lambda a: a.replace("[", "").replace("]", "")\
                                                      .replace(" '", "").replace("'", "")\
                                                      .replace('"', '')\
                                                      .split(","))
ingredients = recipes['ingredients'].apply(pd.Series).stack().reset_index(drop=True)\
.to_frame('ingredients')['ingredients'].value_counts().to_frame('values')

ingredients = ingredients[ingredients['values']>5].to_dict()['values']
clean_ingredients = { k.strip():v for k, v in ingredients.items()}


In [4]:
df = pd.DataFrame (list(clean_ingredients.items()), columns = ['ingredients','total'])
max_value = df["total"].max()
min_value = df["total"].min()
batas_bawah = (max_value - min_value)/3
batas_atas = (max_value - min_value)*2/3
conditions = [
    (df['total'] <= batas_bawah),
    (df['total'] > batas_bawah) & (df['total'] <= batas_atas),
    (df['total'] > batas_atas)
    ]

# create a list of the values we want to assign for each condition
values = ['tinggi', 'sedang', 'rendah']

# create a new column and use np.select to assign values to it using our lists as arguments
df['value'] = np.select(conditions, values)
df.head()


,ingredients,total,value
0,pepper,417,rendah
1,salt,410,rendah
2,garlic,317,rendah
3,water,250,sedang
4,onion,15,tinggi


In [5]:
df.to_csv("../data/ingredients.csv", index=False)